In [155]:
import setup
setup.init_django(project_name='config')

In [156]:
url = "https://www.amazon.com.br/Barra-Prote%C3%ADna-BOLD-Crunch-Cookies/dp/B0D5JJ2KDP/ref=pd_rhf_dp_s_pd_crcd_d_sccl_2_6/145-2538442-3375966?pd_rd_w=emFuO&content-id=amzn1.sym.1b2190ca-db28-46be-85f8-55611bc8c041&pf_rd_p=1b2190ca-db28-46be-85f8-55611bc8c041&pf_rd_r=ME15A3221VFE4TDDADFT&pd_rd_wg=JJ6ft&pd_rd_r=c9fd334d-45f2-4371-8198-0f2e370dd11b&pd_rd_i=B0D5JJ2KDP&psc=1"

In [157]:
!pwd

/home/pittezr-nb/Documents/work/web-scraper/src


In [158]:
import helpers

In [159]:
html = helpers.scrape(url=url)

Connecting to Scraping Browser...
Connected! Navigating to https://www.amazon.com.br/Barra-Prote%C3%ADna-BOLD-Crunch-Cookies/dp/B0D5JJ2KDP/ref=pd_rhf_dp_s_pd_crcd_d_sccl_2_6/145-2538442-3375966?pd_rd_w=emFuO&content-id=amzn1.sym.1b2190ca-db28-46be-85f8-55611bc8c041&pf_rd_p=1b2190ca-db28-46be-85f8-55611bc8c041&pf_rd_r=ME15A3221VFE4TDDADFT&pd_rd_wg=JJ6ft&pd_rd_r=c9fd334d-45f2-4371-8198-0f2e370dd11b&pd_rd_i=B0D5JJ2KDP&psc=1
Navigated! Scraping page content...


In [160]:
from bs4 import BeautifulSoup

In [161]:
soup = BeautifulSoup(html, 'html.parser')

```html
<div id="titleSection" class="a-section a-spacing-none">
    <h1 id="title" class="a-size-large a-spacing-none">
        <span id="productTitle" class="a-size-large product-title-word-break">
            Barra de Proteína BOLD Crunch Cookies &amp; Cream (15g de Proteína) - Caixa com 12 unidades
        </span>
    </h1>
<div id="expandTitleToggle" class="a-section a-spacing-none expand aok-hidden"></div>  </div>
```

In [162]:
productTitle = soup.find('span', id='productTitle')
productTitleText = f"{productTitle.text}".strip()
productTitleText

'Barra de Proteína BOLD Crunch Cookies & Cream (15g de Proteína) - Caixa com 12 unidades'

```html
<span class="a-price-whole">152<span class="a-price-decimal">,</span></span>
```

In [163]:
productPrice = soup.find_all('span', class_='a-price-whole')[0]
productPriceText = "".join([x for x in f"{productPrice.text}".strip() if x.isdigit()] or x == ',')
productPriceNum = float(productPriceText)
productPriceNum

143.0

In [164]:
product_data = soup.find('div', id='prodDetails')
table = product_data.find('table')
columns = [f"{x.text}".strip() for x in table.find_all('th')]
columns

['Informações sobre alergia',
 'Unidades',
 'Marca',
 'Número de unidades',
 'Fabricante',
 'Dimensões do produto',
 'ASIN',
 'Funciona a bateria ou pilha?',
 'EAN']

In [165]:
table_data=[]
for i, row in enumerate(table.find_all('tr')):
    # get all cells in the row
    cells = row.find_all('td')
    # create a dict for the current row, mapping header to cell data
    row_data = {columns[i]: f"{cell.text}".strip() for cell in cells}
    # add the dict to your list
    table_data.append(row_data)

In [166]:
table_data

[{'Informações sobre alergia': '\u200eAmendoim, Nozes, Castanha de caju, Avelãs, Castanha do Brasil'},
 {'Unidades': '\u200e1.00 Unidade'},
 {'Marca': '\u200eBOLD Snacks'},
 {'Número de unidades': '\u200e12'},
 {'Fabricante': '\u200eBOLD Snacks'},
 {'Dimensões do produto': '\u200e16,5 x 13,5 x 8 cm; 650 g'},
 {'ASIN': '\u200eB0D5JJ2KDP'},
 {'Funciona a bateria ou pilha?': '\u200eNão'},
 {'EAN': '\u200e7898755180541'}]

In [167]:
elements_with_attribute = soup.find_all(lambda tag: tag.has_attr('data-csa-c-asin'))
asins = [x.attrs.get('data-csa-c-asin') for x in elements_with_attribute if x]
asins = list(set([x for x in asins if x != ""]))
asins

['B0D5JJ2KDP']

In [168]:
if soup.find('div', id='productDescription'):
    productDescription = soup.find('div', id='productDescription').text

productDescription

' \n SUPLEMENTO ALIMENTAR DE PROTEÍNA EM BARRA  '

In [169]:
featureBullets = soup.find('div', id='feature-bullets').text
featureBullets

' \n Sobre este item    15g de proteínas    Não Contém Glúten    Zero adição de açúcar    Rica em fibras    \n ›  Ver mais detalhes do produto   '

In [170]:
if soup.find(id='averageCustomerReviews'):
    average_rating = soup.find(id='averageCustomerReviews').find_all("span", class_='a-size-base')[0].text.strip()
    average_rating = "".join([x for x in f"{average_rating}".strip() if x.isdigit() or x == ','])
    average_rating = float(average_rating.replace(',', '.'))

(average_rating)

4.7

In [171]:
rating_data = soup.find(id='acrCustomerReviewText').text
rating_count = int("".join([x for x in rating_data if x.isdigit()]))
rating_count

8